## Analyze whether SNWD varies more from year to year or from place to place.

In [23]:
import pandas as pd
import numpy as np
import urllib
import math

In [24]:
import findspark
findspark.init()

from pyspark import SparkContext
sc.stop()
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStats.py'])

from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [25]:
import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
from spark_PCA import computeCov
from computeStats import computeOverAllDist, STAT_Descriptions

In [26]:
### Read the data frame from pickle file

data_dir='../../Data/Weather'
file_index='SSSBSBBS'
meas='SNWD'

from pickle import load

#read statistics
filename=data_dir+'/STAT_%s.pickle'%file_index
STAT,STAT_Descriptions = load(open(filename,'rb'))
print('keys from STAT=',STAT.keys())

#!ls -ld $data_dir/*.parquet

#read data
filename=data_dir+'/decon_%s_%s.parquet'%(file_index,meas)

df=sqlContext.read.parquet(filename)
print(df.count())
df.show(2)

('keys from STAT=', ['TMIN', 'TOBS', 'TMAX', 'SNOW', 'SNWD', 'PRCP'])
330
+-----------------+-------------------+-------------------+---------+--------+--------+---------+-----------+-------------------+-------------------+-------------------+------------------+-----------+------------+------+--------------------+------+
|          coeff_1|            coeff_2|            coeff_3|elevation|   label|latitude|longitude|measurement|              res_1|              res_2|              res_3|          res_mean|    station|   total_var|undefs|              vector|  year|
+-----------------+-------------------+-------------------+---------+--------+--------+---------+-----------+-------------------+-------------------+-------------------+------------------+-----------+------------+------+--------------------+------+
|5023.264685707683| 1366.1120680592153|-1450.7536366819663|   2235.4|SSSBSBBS| 38.9783|-119.8914|       SNWD|0.24744997302435823|0.22476502263087722| 0.1838983630669596|0.85622048

In [27]:
print df.columns

['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'label', 'latitude', 'longitude', 'measurement', 'res_1', 'res_2', 'res_3', 'res_mean', 'station', 'total_var', 'undefs', 'vector', 'year']


In [28]:
#extract longitude and latitude for each station
feature='coeff_1'
sqlContext.registerDataFrameAsTable(df,'weather')
#Features=', '.join(['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'latitude', 'longitude',\
#          'res_1', 'res_2', 'res_3', 'res_mean', 'year'])
Features='station, year, coeff_2'
Query="SELECT %s FROM weather"%Features
print(Query)
pdf = sqlContext.sql(Query).toPandas()
pdf.head()

SELECT station, year, coeff_2 FROM weather


station    year      coeff_2
0  USC00262119  2010.0  1366.112068
1  USC00042671  1981.0 -1989.935037
2  USC00045571  1964.0   397.956945
3  USC00049105  1951.0  -773.355221
4  USC00049105  1954.0 -2587.184031

In [29]:
year_station_table=pdf.pivot(index='year', columns='station', values='coeff_2')
year_station_table.head(10)

station  USC00040943  USC00041072  USC00041075  USC00041277  USC00041909  \
year                                                                       
1915.0           NaN   345.601057          NaN          NaN          NaN   
1932.0           NaN          NaN          NaN          NaN          NaN   
1933.0           NaN          NaN          NaN          NaN          NaN   
1937.0           NaN          NaN          NaN          NaN          NaN   
1948.0           NaN          NaN          NaN          NaN          NaN   
1949.0           NaN          NaN  1116.600817          NaN   678.788401   
1950.0           NaN          NaN   125.702280          NaN          NaN   
1951.0           NaN          NaN  -163.868047          NaN          NaN   
1952.0           NaN          NaN          NaN          NaN          NaN   
1953.0           NaN          NaN          NaN          NaN          NaN   

station  USC00041911  USC00042671  USC00043134  USC00043381  USC00043384  \
year                                                                       
1915.0           NaN          NaN          NaN          NaN          NaN   
1932.0           NaN          NaN          NaN          NaN          NaN   
1933.0           NaN          NaN          NaN          NaN          NaN   
1937.0           NaN          NaN          NaN          NaN          NaN   
1948.0           NaN          NaN          NaN          NaN          NaN   
1949.0           NaN          NaN   432.585614   383.415811   906.866851   
1950.0           NaN          NaN   498.219868   295.918668   711.900418   
1951.0           NaN          NaN   146.708225          NaN    59.785000   
1952.0    505.661215          NaN  1044.930075   256.993663   951.651317   
1953.0           NaN          NaN          NaN          NaN          NaN   

station     ...       USC00263515  USC00265168  USC00265191  USC00266746  \
year        ...                                                            
1915.0      ...               NaN          NaN          NaN          NaN   
1932.0      ...               NaN          NaN    66.963757          NaN   
1933.0      ...               NaN   360.630168   299.453499          NaN   
1937.0      ...               NaN   345.240165          NaN          NaN   
1948.0      ...               NaN          NaN          NaN          NaN   
1949.0      ...        337.125734          NaN   645.231023          NaN   
1950.0      ...               NaN          NaN          NaN          NaN   
1951.0      ...               NaN          NaN          NaN          NaN   
1952.0      ...               NaN          NaN          NaN          NaN   
1953.0      ...               NaN          NaN          NaN          NaN   

station  USC00267640  USC00267806  USC00268186  USC00268202  USC00268977  \
year                                                                       
1915.0           NaN          NaN          NaN          NaN          NaN   
1932.0           NaN          NaN          NaN          NaN          NaN   
1933.0           NaN          NaN          NaN          NaN          NaN   
1937.0           NaN          NaN          NaN          NaN          NaN   
1948.0           NaN          NaN          NaN          NaN          NaN   
1949.0           NaN          NaN          NaN          NaN   160.273339   
1950.0           NaN          NaN          NaN          NaN          NaN   
1951.0           NaN          NaN          NaN          NaN          NaN   
1952.0           NaN          NaN          NaN          NaN          NaN   
1953.0           NaN          NaN          NaN          NaN          NaN   

station  USC00269229  
year                  
1915.0           NaN  
1932.0           NaN  
1933.0           NaN  
1937.0           NaN  
1948.0           NaN  
1949.0    274.734895  
1950.0           NaN  
1951.0           NaN  
1952.0           NaN  
1953.0           NaN  

[10 rows x 36 columns]

### Estimating the effect of the year vs the effect of the station

To estimate the effect of time vs. location on the first eigenvector coefficient we
compute:

* The average row: `mean-by-station`
* The average column: `mean-by-year`

We then compute the RMS before and after subtracting either  the row or the column vector.

In [30]:
def RMS(Mat):
    return np.sqrt(np.nanmean(Mat**2))

mean_by_year=np.nanmean(year_station_table,axis=1)
mean_by_station=np.nanmean(year_station_table,axis=0)
tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
tbl_minus_station = year_station_table-mean_by_station

print 'total RMS                   = ',RMS(year_station_table)
print 'RMS removing mean-by-station= ',RMS(tbl_minus_station)
print 'RMS removing mean-by-year   = ',RMS(tbl_minus_year)

total RMS                   =  1174.86213425
RMS removing mean-by-station=  978.857625043
RMS removing mean-by-year   =  882.700935227


In [31]:
T=year_station_table
print 'initial RMS=',RMS(T)
for i in range(5):
    mean_by_year=np.nanmean(T,axis=1)
    T=(T.transpose()-mean_by_year).transpose()
    print i,'after removing mean by year    =',RMS(T)
    mean_by_station=np.nanmean(T,axis=0)
    T=T-mean_by_station
    print i,'after removing mean by stations=',RMS(T)

initial RMS= 1174.86213425
0 after removing mean by year    = 882.700935227
0 after removing mean by stations= 766.35037481
1 after removing mean by year    = 742.904757935
1 after removing mean by stations= 736.80380624
2 after removing mean by year    = 734.770786935
2 after removing mean by stations= 733.904522002
3 after removing mean by year    = 733.452326169
3 after removing mean by stations= 733.182840879
4 after removing mean by year    = 733.010075457
4 after removing mean by stations= 732.895056638
